In [1]:
notebook_name = 'main'
version = 16

Copyright 2023,
Tom Wenk, Julian Kartte, Markus Siebertz,
All rights reserved.

# Project v16 Improvements based on v15:
- Train activities with Passengers:
    - Trains are sorted and processed by the expected arrival of their Passengers instead of their speed
    (Trains without Passengers are still processed ordered by their speed)
- When searching for an empty Train for an exchange at a Station:
    - If a Train was found and he already has an active route, the route is adapted to execute the exchange
    and continue on its old route after that

## FP Project Team:
- Markus Siebertz
- Julian Kartte
- Tom Wenk

## ----------------!!!!!!!!!!!!!!!!!----------------
To execute the program, please make sure that an input file is correctly referenced in the function read_input_from_file
## ----------------!!!!!!!!!!!!!!!!!----------------

## Further possible improvements:
- Trains can drive through a Station if no passenger wants to leave or board
- For passengers that are not first priority of the fastest Train at its current Station, check if taking other trains might be faster
- Prioritizing passengers should also take the length to their destination into account
- Trains without passengers should check if they are of use at another station or if they just block the line, before they depart without passengers
- Complete different approach (Machine Learning algorithms, ...)

### Algorithm:
0. Preparation
    - Sort passengers by expected arrival
    - Sort trains by their speed
    - Determine the shortest route for each passenger from current location
    - Set starting station for every train that has none
1. Increment round
2. Execute actions (capacities can be temporarily exceeded)
3. Check capacities at the end of each round
4. Check end conditions at the end of each round
5. Back to 1. (if not ended)

### Actions:
- Stations
- Lines
- Passengers
    1. Board
    2. Detrain
- Train
    1. Depart (after checking capacity)
    2. Arrive (if speed x round > line length)

### End Conditions:
- All passengers arrived at their destination
- At the end of the round all capacities are kept

# Import external Libraries

In [2]:
import sys                  # for input over command line
from typing import List     # for the use of Lists
from pathlib import Path    # for independence from operation system when working with file paths
import traceback            # for printing to console in case of an error
import pandas as pd         # for the use of DataFrames
import os                   # for accessing local file system

# Prepare Functions and Classes

## Classes and Functions for Dijkstra-algorithm

In [3]:
# code is inspired by: https://www.udacity.com/blog/2021/10/implementing-dijkstras-algorithm-in-python.html

class Graph(object):
    def __init__(self, nodes, init_graph):
        self.__nodes = nodes
        self.__graph = self.__construct_graph(nodes, init_graph)

    @staticmethod
    def __construct_graph(nodes, init_graph):
        """
        This method makes sure that the graph is symmetrical. In other words, if there's a path from node A to B with a value V,
        there needs to be a path from node B to node A with a value V.
        """
        graph = {}
        for node in nodes:
            graph[node] = {}

        graph.update(init_graph)

        for node, edges in graph.items():
            for adjacent_node, value in edges.items():
                if not graph[adjacent_node].get(node, False):
                    graph[adjacent_node][node] = value

        return graph

    def get_nodes(self):
        """Returns the nodes of the graph."""
        return self.__nodes

    def get_outgoing_edges(self, node):
        """Returns all neighbors of a node."""
        connections = []
        for out_node in self.__nodes:
            if self.__graph[node].get(out_node, False):
                connections.append(out_node)
        return connections

    def value(self, node1, node2):
        """Returns the value (weight) of an edge between two nodes."""
        return self.__graph[node1][node2]

def dijkstra_algorithm(graph: Graph, start_node: str):
    """Calculates shortest paths with costs in passed graph to each other nodes from the passed start node

    :param graph: Graph that represents the network
    :param start_node: starting node
    :return: 
    """
    unvisited_nodes = list(graph.get_nodes())

    # We'll use this dict to save the cost of visiting each node and update it as we move along the graph
    shortest_path = {}

    # We'll use this dict to save the shortest known path to a node found so far
    previous_nodes = {}

    # We'll use max_value to initialize the "infinity" value of the unvisited nodes
    max_value = sys.maxsize
    for node in unvisited_nodes:
        shortest_path[node] = max_value
    # However, we initialize the starting node's value with 0
    shortest_path[start_node] = 0

    # The algorithm executes until we visit all nodes
    while unvisited_nodes:
        # The code block below finds the node with the lowest score
        current_min_node = None
        for node in unvisited_nodes: # Iterate over the nodes
            if current_min_node is None:
                current_min_node = node
            elif shortest_path[node] < shortest_path[current_min_node]:
                current_min_node = node

        # The code block below retrieves the current node's neighbors and updates their distances
        neighbors = graph.get_outgoing_edges(current_min_node)
        for neighbor in neighbors:
            tentative_value = shortest_path[current_min_node] + graph.value(current_min_node, neighbor)
            if tentative_value < shortest_path[neighbor]:
                shortest_path[neighbor] = tentative_value
                # We also update the best path to the current node
                previous_nodes[neighbor] = current_min_node

        # After visiting its neighbors, we mark the node as "visited"
        unvisited_nodes.remove(current_min_node)

    return previous_nodes, shortest_path

# For manual testing during development
def print_result(previous_nodes, shortest_path, start_node, target_node):
    path = []
    node = target_node

    while node != start_node:
        path.append(node)
        node = previous_nodes[node]

    # Add the start node manually
    path.append(start_node)

    print("Test Output:")
    print("We found the following best path with a value of {}.".format(shortest_path[target_node]))
    print(" -> ".join(reversed(path)))

## Classes and functions for objects and their interaction

In [4]:
class Station(object):
    """
    A Station holds Trains and Passengers and is connected to other Stations over Lines.

    Passengers and Trains can leave or arrive at Stations.
    """
    def __init__(self, identifier: str, capacity: int):
        self.__id: str = identifier
        self.__capacity: int = capacity
        self.__trains_in_station: List[str] = []
        self.__passengers_in_station: List[str] = []
        self.__waiting_passengers_in_station: List[str] = []
        self.__lines_from_station: List[str] = []

    def get_id(self):
        """Returns the ID of the Station
        :return: ID of the Station
        """
        return self.__id

    def get_capacity(self):
        """
        Returns the capacity of this Station
        :return: Capacity of this Station
        """
        return self.__capacity

    def get_remaining_capacity(self):
        """
        Returns remaining capacity of Trains for this Station
        :return: Remaining capacity of Trains for this Station (capacity - trains in station)
        """
        return self.__capacity - len(self.__trains_in_station)

    def get_trains_in_station(self):
        """
        Returns list of IDs from Trains that are currently at this Station
        :return: List of IDs from Trains that are currently at this Station
        """
        return self.__trains_in_station

    def get_passengers_in_station(self):
        """
        Returns list of IDs from Passengers that are currently at this Station
        :return: List of IDs from Passengers that are currently at this Station
        """
        return self.__passengers_in_station

    def get_waiting_passengers_in_station(self):
        """
        Returns list of IDs from Passengers that are currently at this Station, which is not their destination
        :return: List of IDs from Passengers that are currently at this Station, that want to leave
        """
        return self.__waiting_passengers_in_station

    def get_lines_from_station(self):
        """
        Returns list of IDs from Lines that are connected to this Station
        :return: List of IDs from Lines that are connected to this Station
        """
        return self.__lines_from_station

    def set_lines_from_station(self, line_ids: [str]):
        """
        Replaces the complete list of IDs from Lines that are connected to this Station
        :param line_ids: Replace list of IDs from Lines that are connected to this Station with passed list
        :return:
        """
        self.__lines_from_station = line_ids

    def add_line_to_lines_from_station(self, line_id: str):
        """
        Add passed Line ID to list of Lines connected to this Station
        :param line_id:
        :return:
        """
        if line_id not in self.__lines_from_station:
            self.__lines_from_station.append(line_id)

    def train_arrives(self, train_id):
        """
        Add Train ID to list of Trains in this Station
        :param train_id:
        :return:
        """
        self.__trains_in_station.append(train_id)

    def train_departs(self, train_id):
        """
        Remove Train ID from list of Trains in this Station
        :param train_id:
        :return:
        """
        try:
            self.__trains_in_station.remove(train_id)
        except:
            print(str(current_round), 'Failed to remove', train_id, 'from', self.__id)

    def passenger_boards(self, passenger_id):
        """
        Remove Passenger ID from list of Passengers in this Station
        :param passenger_id:
        :return:
        """
        self.__passengers_in_station.remove(passenger_id)
        self.__waiting_passengers_in_station.remove(passenger_id)

    def passenger_detrains(self, passenger_id):
        """
        Add Passenger ID to list of Passengers in this Station
        :param passenger_id:
        :return:
        """
        self.__passengers_in_station.append(passenger_id)
        self.__waiting_passengers_in_station.append(passenger_id)

    def passenger_arrives(self, passenger_id):
        """
        Add Passenger ID to list of Passengers in this Station
        :param passenger_id:
        :return:
        """
        self.__passengers_in_station.append(passenger_id)

class Line(object):
    """
    A Line holds Trains and connects two Stations in both directions.

    Trains can start on or leave a Line.
    """
    def __init__(self, identifier: str, first_station_id: str, second_station_id: str, length: float, capacity:int):
        self.__id: str = identifier
        self.__first_station: str = first_station_id
        self.__second_station: str = second_station_id
        self.__length: float = length
        self.__capacity: int = capacity
        self.__trains_on_line: List[str] = []

    def get_id(self):
        """Returns the ID of the Line"""
        return self.__id

    def get_first_station(self):
        """Returns the ID of the first Station connected by the Line"""
        return self.__first_station

    def get_second_station(self):
        """Returns the ID of the second Station connected by the Line"""
        return self.__second_station

    def get_length(self):
        """Returns length of Line"""
        return self.__length

    def get_capacity(self):
        """Returns total train-capacity of Line"""
        return self.__capacity

    def get_remaining_capacity(self):
        """Returns remaining train-capacity of Line"""
        return self.__capacity - len(self.__trains_on_line)

    def get_trains_on_line(self):
        """Returns list of IDs of Trains on the Line"""
        return self.__trains_on_line

    def train_enters(self, train_id):
        """
        Appends passed Train-ID to list of Trains currently on this Line.
        :param train_id:
        :return:
        """
        self.__trains_on_line.append(train_id)

    def train_leaves(self, train_id):
        """
        Remove passed Train-ID from list of Trains currently on this Line.
        :param train_id:
        :return:
        """
        self.__trains_on_line.remove(train_id)

class Train(object):
    """
    A Train holds Passengers and drives on Lines from Station to Station.

    Passengers can board or leave a Train.
    """
    def __init__(self, identifier: str, start_station_id: str, speed: float, capacity: int, graph: Graph):
        self.__id: str = identifier
        self.__start_station_id: str = start_station_id
        self.__speed: float = speed
        self.__capacity: int = capacity
        self.__next_station_id: str = ''
        self.__current_location_id: str = start_station_id
        self.__time_on_current_location: int = 0
        self.__passengers_on_train: List[str] = []
        self.__passenger_utilization: int = 0
        self.__route: List[str] = []
        self.__priority_station_id: str = ''
        self.__graph: Graph = graph

    def get_id(self):
        """
        Returns ID of Train.
        :return:
        """
        return self.__id

    def get_start_station_id(self):
        return self.__start_station_id

    def get_speed(self):
        return self.__speed

    def get_capacity(self):
        return self.__capacity

    def get_remaining_capacity(self):
        return self.__capacity - self.__passenger_utilization

    def get_passenger_utilization(self):
        return self.__passenger_utilization

    def get_current_location_id(self):
        return self.__current_location_id

    def get_passengers_on_train(self):
        return self.__passengers_on_train

    def get_time_on_current_location(self):
        return self.__time_on_current_location

    def get_next_station_id(self):
        return self.__next_station_id

    def get_priority_station_id(self):
        return self.__priority_station_id

    def get_route(self):
        return self.__route

    def get_graph(self):
        return self.__graph

    def set_next_station_id(self, station_id: str):
        self.__next_station_id = station_id

    def set_priority_station_id(self, station_id: str):
        self.__priority_station_id = station_id

    def set_route_manually(self, new_route: List[str]):
        self.__route = new_route

    def set_route(self):
        """
        Execute Dijkstra Algorithm to find fastest path between current location and every other station.

        :return:
        """
        start_station_id = self.__current_location_id
        # path_costs = dictionary
        #   ID:     destination ID,
        #   Value:  costs for shortest path
        # prev_nodes = dictionary
        #   ID:     destination ID,
        #   Value:  next Station/node on reversed path (from destination to start)
        prev_nodes, path_costs = dijkstra_algorithm(graph = self.__graph, start_node = start_station_id)
        temp_path = []
        node = self.__priority_station_id # Start with destination as current node

        while node != start_station_id:
            # Build path/route from destination to current (start) location
            temp_path.append(node)
            node = prev_nodes[node]

        # reverse path from destination to start to get route from start to destination
        self.__route = list(reversed(temp_path))
        if len(self.__route) > 0:
            # If route has successfully been set, next station can be read from route
            self.__next_station_id = self.__route[0]

    def set_starting_station(self, station_id: str):
        if self.__start_station_id == '*':
            self.__start_station_id = station_id
            self.__current_location_id = station_id

    def depart_on_line(self, line: Line):
        """
        Replaces current location of Train with ID of passed Line object.

        Replace next station ID by the station from the passed line, that is not this station.

        Reset time on current location.

        :param line: Line object that the Train departs on.
        :return:
        """
        current_station = next((station for station in stations if station.get_id() == self.__current_location_id), '')
        if current_station == '':
            file_log.write('\n\nERROR: Train ' + self.__id + ' is not at a Station, so he can not depart!\n')
            print('\n-------------------------------')
            print('------------ ERROR ------------')
            print('Train', self.__id, ' is not in a Station, so he can not depart!')
            print('-------------------------------\n')
            return
        if line.get_first_station() == current_station.get_id():
            self.__next_station_id = line.get_second_station()
        elif line.get_second_station() == current_station.get_id():
            self.__next_station_id = line.get_first_station()

        current_station.train_departs(self.__id)
        self.__current_location_id = line.get_id()
        line.train_enters(self.__id)
        self.__time_on_current_location = 0

    def arrive_in_station(self, station: Station):
        """
        Remove Train from current Line.

        Remove ID of passed Station from route.

        Add Train ID to List of Trains at passed Station.
        :param station:
        :return:
        """
        try:
            current_line = next((line for line in lines if line.get_id() == self.__current_location_id), '')
            #print(self.__id, self.__current_location_id, station.get_id(), self.__route)
            if current_line == '':
                file_log.write('\n\nERROR: Train ' + self.__id + ' is not on a Line, so he can not arrive at Station ' + station.get_id() + '!\n')
                print('\n-------------------------------')
                print('------------ ERROR ------------')
                print('Train', self.__id, 'is not on a Line (', self.__current_location_id, '), so he can not arrive in Station', station.get_id(), '!')
                print('-------------------------------\n')
                return

            current_line.train_leaves(self.__id)
            if len(self.__route) > 0:
                self.__route.remove(station.get_id())
            self.__current_location_id = station.get_id()
            #print(self.__route)
            if len(self.__route) > 0:
                self.__next_station_id = self.__route[0]
            station.train_arrives(self.__id)
            self.__time_on_current_location = 0
        except:
            print('\nError for Train:', self.__id)
            print('Passed Station:', station.get_id())
            print('Train Route:', self.__route)
            raise

    def passenger_boards(self, passenger_id: str, passenger_group_size: int):
        self.__passengers_on_train.append(passenger_id)
        self.__passenger_utilization = self.__passenger_utilization + passenger_group_size

    def passenger_detrains(self, passenger_id: str, passenger_group_size: int):
        self.__passengers_on_train.remove(passenger_id)
        self.__passenger_utilization = self.__passenger_utilization - passenger_group_size

    def all_passengers_detrain(self):
        for passenger_id in self.__passengers_on_train:
            self.__passengers_on_train.remove(passenger_id)

    def increment_time_on_current_location(self):
        self.__time_on_current_location = self.__time_on_current_location + 1

    def calculate_duration_multiple_routes(self, end_station_id: str, expected_arrival: int, current_passenger_id: str):
        # return values
        calculated_arrival: int = sys.maxsize
        total_duration_diff_all_passengers: int
        total_delay_diff_all_passengers: int

        # variables
        total_duration_with_new_passenger: int
        total_duration_without_new_passenger: int
        total_delay_with_new_passenger: int
        total_delay_without_new_passenger: int

        start_station_id = self.__current_location_id

        # Calculate duration to get all passengers on current train to their destination
        temp_total_path = []
        temp_sum_delay = 0 # Delay of all Passengers
        temp_sum_duration = 0 # Duration to endstation of passed/current Passenger
        temp_vip_route = []
        for vip in passengers:
            if vip.get_id() not in self.__passengers_on_train:
                # Passengers that are not on the train are skipped
                continue

            if vip.get_end_station_id() in temp_total_path:
                # Path costs of Passenger that was not VIP and detrained on another Passenger's
                # path should not be calculated multiple times
                if vip.get_end_station_id() != temp_total_path[-1]:
                    # Leaving the Train costs one extra round, if it is not the current station
                    temp_sum_duration = temp_sum_duration + 1
                continue

            # Calculate fastest paths from passed start station to every other station
            prev_nodes, path_costs = dijkstra_algorithm(graph = self.__graph, start_node = start_station_id)

            temp_current_path = []
            node = vip.get_end_station_id() # Start with destination as current node

            while node != start_station_id:
                # Build path/route from destination to current (start) location
                temp_current_path.append(node)
                node = prev_nodes[node]

            # reverse path from destination to start to get route from start to destination
            temp_vip_route = list(reversed(temp_current_path))
            temp_total_path = temp_total_path + temp_vip_route

            if expected_arrival >= vip.get_expected_arrival() \
                    and end_station_id in temp_total_path:
                # If end station of new Passenger is part of the route the train takes when delivering other Passengers with higher need
                # there will be no delay through the boarding of the new Passenger
                temp_sum_duration = temp_sum_duration + path_costs[end_station_id]
                total_duration_diff_all_passengers = 0
                total_delay_diff_all_passengers = 0
                calculated_arrival = temp_sum_duration
                return calculated_arrival, total_duration_diff_all_passengers, total_delay_diff_all_passengers

            # Add duration to get to destination of current Passenger to total sum
            temp_sum_duration = temp_sum_duration + path_costs[vip.get_end_station_id()] + 1
            if temp_sum_duration > vip.get_expected_arrival():
                # Add delay to total sum, if any appears
                temp_sum_delay = temp_sum_delay + temp_sum_duration - vip.get_expected_arrival()
            start_station_id = vip.get_end_station_id()

        total_duration_without_new_passenger = temp_sum_duration
        total_delay_without_new_passenger = temp_sum_delay

        # Calculate duration to get all passengers on current train and the additional passenger to their destination
        temp_total_path = []
        temp_sum_delay = 0 # Delay of all Passengers
        temp_sum_duration = 0 # Duration to endstation of passed/current Passenger
        temp_vip_route = []
        for vip in passengers:
            if vip.get_id() not in self.__passengers_on_train:
                # Passengers that are not on the train are skipped
                # except for the current one that may want to board
                if vip.get_id() != current_passenger_id:
                    continue

            if vip.get_end_station_id() in temp_total_path:
                # Path costs of Passenger that was not VIP and detrained on another Passenger's
                # path should not be calculated multiple times
                if vip.get_end_station_id() != temp_total_path[-1]:
                    # Leaving the Train costs one extra round, if it is not the current station
                    temp_sum_duration = temp_sum_duration + 1
                continue

            # Calculate fastest paths from passed start station to every other station
            prev_nodes, path_costs = dijkstra_algorithm(graph = self.__graph, start_node = start_station_id)

            temp_current_path = []
            node = vip.get_end_station_id() # Start with destination as current node

            while node != start_station_id:
                # Build path/route from destination to current (start) location
                temp_current_path.append(node)
                node = prev_nodes[node]

            # reverse path from destination to start to get route from start to destination
            temp_vip_route = list(reversed(temp_current_path))
            temp_total_path = temp_total_path + temp_vip_route

            if end_station_id in temp_total_path:
                # If end station of new Passenger is part of the route the train takes when delivering other Passengers with higher need
                # there will be no delay through the boarding of the new Passenger
                calculated_arrival = temp_sum_duration + path_costs[end_station_id]

            # Add duration to get to destination of current Passenger to total sum
            temp_sum_duration = temp_sum_duration + path_costs[vip.get_end_station_id()] + 1
            if temp_sum_duration > vip.get_expected_arrival():
                temp_sum_delay = temp_sum_delay + temp_sum_duration - vip.get_expected_arrival()
            start_station_id = vip.get_end_station_id()

        total_duration_with_new_passenger = temp_sum_duration
        total_delay_with_new_passenger = temp_sum_delay

        total_duration_diff_all_passengers = total_duration_with_new_passenger - total_duration_without_new_passenger
        total_delay_diff_all_passengers = total_delay_with_new_passenger - total_delay_without_new_passenger

        return calculated_arrival, total_duration_diff_all_passengers, total_delay_diff_all_passengers

    def calculate_duration_current_route(self, end_station_id: str):
        """
        Calculates duration from current location to passed destination.
        :param end_station_id:
        :return:
        """
        start_station_id = self.__current_location_id

        # path_costs = dictionary
        #   ID:     destination ID,
        #   Value:  costs for shortest path
        # prev_nodes = dictionary
        #   ID:     destination ID,
        #   Value:  next Station/node on reversed path (from destination to start)
        prev_nodes, path_costs = dijkstra_algorithm(graph = self.__graph, start_node = start_station_id)

        return path_costs[end_station_id]

    def calculate_duration_fastest_route(self, end_station_id: str = None, start_station_id: str = None):
        """
        Execute Dijkstra Algorithm to find fastest path between current location and every other station.
        :return:
        """
        if start_station_id == None:
            start_station_id = self.__current_location_id
        if end_station_id == None:
            end_station_id = self.__priority_station_id

        # path_costs = dictionary
        #   ID:     destination ID,
        #   Value:  costs for shortest path
        # prev_nodes = dictionary
        #   ID:     destination ID,
        #   Value:  next Station/node on reversed path (from destination to start)
        prev_nodes, path_costs = dijkstra_algorithm(graph = self.__graph, start_node = start_station_id)

        return path_costs[end_station_id]

class Passenger(object):
    def __init__(self, identifier: str, start_station_id: str, end_station_id: str, group_size: int, expected_arrival: int):
        self.__id: str = identifier
        self.__start_station_id: str = start_station_id
        self.__end_station_id: str = end_station_id
        self.__group_size: int = group_size
        self.__expected_arrival: int = expected_arrival
        self.__current_location_id: str = start_station_id
        self.__delay: int = -1

    def get_id(self):
        return self.__id

    def get_start_station_id(self):
        return self.__start_station_id

    def get_end_station_id(self):
        return self.__end_station_id

    def get_group_size(self):
        return self.__group_size

    def get_expected_arrival(self):
        return self.__expected_arrival

    def get_current_location_id(self):
        return self.__current_location_id

    def get_delay(self):
        return self.__delay

    def board(self, train: Train):
        current_station = next((station for station in stations if station.get_id() == self.__current_location_id), '')
        if current_station == '':
            file_log.write('\n\nERROR: Passenger ' + self.__id + ' is not at a Station, so he can not board a Train!\n')
            print('\n-------------------------------')
            print('------------ ERROR ------------')
            print('Passenger is not in a Station, so he can not board a Train!')
            print('-------------------------------\n')
            return

        current_station.passenger_boards(self.__id)
        train.passenger_boards(self.__id, self.__group_size)
        self.__current_location_id = train.get_id()

    def detrain(self, station: Station, curr_round: int):
        curr_train = next((train for train in trains if train.get_id() == self.__current_location_id), '')
        if curr_train == '':
            file_log.write('\n\nERROR: Passenger ' + self.__id + ' is not on a Train, so he can not detrain!\n')
            print('\n-------------------------------')
            print('------------ ERROR ------------')
            print('Passenger is not on a Train, so he can not detrain!')
            print('-------------------------------\n')
            return

        curr_train.passenger_detrains(self.__id, self.__group_size)
        if station.get_id() == self.__end_station_id:
            station.passenger_arrives(self.__id)
            difference = curr_round - self.__expected_arrival
            self.__delay = difference if difference > 0 else 0
            file_log.write('\n! Passenger ' + str(self.__id) + ' arrived at its final destination on round ' + str(current_round) + ' with a delay of ' + str(self.__delay) + ' rounds')
            #print('! Passenger', self.__id, 'arrived at its final destination on round', curr_round, 'with a delay of', self.__delay, 'rounds')
        else:
            station.passenger_detrains(self.__id)
        self.__current_location_id = station.get_id()

## Functions for preprocessing and preparation

Non-object-functions used for:
- data preprocessing
- algorithm preparation
- better readability & understandability of the code

In [5]:
def data_preprocessing(data):
    """
    Clean up, convert and split data for further processing.

    :param data: List of strings representing line-by-line content of input text file
    :return: 4 DataFrames containing split data from input (stations, lines, trains, passengers)
    """

    # Clean up data
    data = map(str.rstrip, data)   # remove \n at the end of each string (row/line) in list
    df = pd.DataFrame(data, columns = ['data'])   # convert list of strings into DataFrame with column name 'data'
    df = df.loc[df['data'] != '']   # remove empty rows
    df = df.loc[(df['data'].str.contains('#') == False)]   # remove comment lines/rows (containing #)
    df = df.reset_index(drop = True)   # reset indexes in DataFrame to be continuous again after removing rows/lines

    # Extract indexes:
    station_index = df.index[df['data'] == '[Stations]'][0]
    lines_index = df.index[df['data'] == '[Lines]'][0]
    trains_index = df.index[df['data'] == '[Trains]'][0]
    passengers_index = df.index[df['data'] == '[Passengers]'][0]

    # Split data into DataFrames
    station_df = pd.DataFrame(df['data'].loc[station_index + 1 : lines_index - 1].str.split(expand = True)) # extract Stations
    station_df.columns = ['ID', 'Capacity'] # Rename columns
    station_df = station_df.reset_index(drop = True) # Reset Indexes
    line_df = pd.DataFrame(df['data'].loc[lines_index + 1 : trains_index - 1].str.split(expand = True)) # extract Lines
    line_df.columns = ['ID', 'StartStation', 'EndStation', 'Length', 'Capacity'] # Rename columns
    line_df = line_df.reset_index(drop = True) # Reset Indexes
    train_df = pd.DataFrame(df['data'].loc[trains_index + 1 : passengers_index - 1].str.split(expand = True)) # extract Trains
    train_df.columns = ['ID', 'StartStation', 'Speed', 'Capacity'] # Rename columns
    train_df = train_df.reset_index(drop = True) # Reset Indexes
    passenger_df = pd.DataFrame(df['data'].loc[passengers_index + 1 : ].str.split(expand = True)) # extract Passengers
    passenger_df.columns = ['ID', 'StartStation', 'Destination', 'GroupSize', 'ExpectedArrival'] # Rename columns
    passenger_df = passenger_df.reset_index(drop = True) # Reset Indexes

    return station_df, line_df, train_df, passenger_df

def create_train_graphs():
    """
    Creates dictionary with one graph for every train.

    The weight of the edges is equal to the amount of rounds a train needs to cover that line.

    :return: Dictionary containing Train IDs and their corresponding graphs.
    """
    # Initialize one graph for every train
    nodes = stations_df.ID.tolist() # Nodes in graph = Station IDs
    train_graph = {} # Dictionary for train-ID and corresponding graph

    for index, train in trains_df.iterrows():
        # Initialize empty graph for every train
        init_graph = {}
        for node in nodes:
            init_graph[node] = {}

        for i in range(0, len(lines_df)):
            # Set the costs of a Line between start Station and end Station of that Line
            line_length = float(lines_df.loc[i]['Length'])
            velocity = float(train['Speed'])
            init_graph[lines_df.loc[i]['StartStation']][lines_df.loc[i]['EndStation']] = int(line_length/velocity) + 1

        # Assign graph as object to the corresponding key in the dictionary train_graph
        graph = Graph(nodes, init_graph)
        train_graph[train.ID] = graph

    return train_graph

def map_df_to_objects(station_df, line_df, train_df, passenger_df):
    """
    Map DataFrames to lists of objects from input.

    :param station_df: DataFrame containing information of all Stations
    :param line_df: DataFrame containing information of all Lines
    :param train_df: DataFrame containing information of all Trains
    :param passenger_df: DataFrame containing information of all Passengers
    :return : 4 lists of objects: Stations, Lines, Trains, Passengers
    """

    if station_df.empty:
        station_df = stations_df
    if line_df.empty:
        line_df = lines_df
    if train_df.empty:
        train_df = trains_df
    if passenger_df.empty:
        passenger_df = passengers_df

    passengers_list = [
        (Passenger(
            row.ID,
            row.StartStation,
            row.Destination,
            int(row.GroupSize),
            int(row.ExpectedArrival)
        )) for index, row in passenger_df.iterrows()
    ]
    lines_list = [
        (Line(
            row.ID,
            row.StartStation,
            row.EndStation,
            float(row.Length),
            int(row.Capacity)
        )) for index, row in line_df.iterrows()
    ]
    trains_list = [
        (Train(
            row.ID,
            row.StartStation,
            float(row.Speed),
            int(row.Capacity),
            train_graphs[row.ID]
        )) for index, row in train_df.iterrows()
    ]
    stations_list = [
        (Station(
            row.ID,
            int(row.Capacity)
         )) for index, row in station_df.iterrows()
    ]

    return stations_list, lines_list, trains_list, passengers_list

def preparation():
    """
    Preparation for the algorithm.

    - Calculating maximum number of rounds for the algorithm to run

    - Sorting Passengers

    - Sorting Trains

    - Determine starting station for Trains

    :return: Maximum number of rounds the algorithm should run before aborting due to possible deadlock (endless loop)
    """
    # Show input / starting situation
    file_log.write('--------------------- Starting Situation (Input) ---------------------')
    #print('--------------------- Starting Situation (Input) ---------------------')
    file_log.write('\nStations:')
    file_log.write('\n' + str(stations_df))
    #print('Stations:')
    #print(stations_df)
    file_log.write('\nLines:')
    file_log.write('\n' + str(lines_df))
    #print('\nLines:')
    #print(lines_df)
    file_log.write('\nTrains:')
    file_log.write('\n' + str(trains_df))
    #print('\nTrains:')
    #print(trains_df)
    file_log.write('\nPassengers:')
    file_log.write('\n' + str(passengers_df))
    #print('\nPassengers:')
    #print(passengers_df)

    file_log.write('\n\n--------------------- Preparation ---------------------')
    #print('\n\n--------------------- Preparation ---------------------')

    maximal_rounds = 0

    # Sort Passengers by their arrival time
    passengers.sort(key=lambda p: p.get_expected_arrival())

    # Sort Trains by their speed
    trains.sort(key=lambda t: t.get_speed(), reverse=True)

    # Add Passengers to the list of Passengers on each Station
    # Determine the latest expected arrival for the emergency break in case of an deadlock in the endless loop
    for p in passengers:
        starting_station = next((station for station in stations if station.get_id() == p.get_current_location_id()), '')
        if starting_station.get_id() == p.get_end_station_id():
            starting_station.passenger_arrives(p.get_id())
        else:
            starting_station.passenger_detrains(p.get_id())
        if p.get_expected_arrival() * 2 > maximal_rounds:
            maximal_rounds = p.get_expected_arrival() * 2

    # Determine starting Stations for Trains without one
    # Add all Trains to the list of Trains of their starting Station
    for train in [t for t in trains if t.get_start_station_id() != '*']:
        starting_station = next((station for station in stations if station.get_id() == train.get_start_station_id()), '')
        starting_station.train_arrives(train.get_id())

    for train in [t for t in trains if t.get_start_station_id() == '*']:
        biggest_diff_passengers_capacities = -sys.maxsize

        station_highest_need_of_train: Station
        if 'station_highest_need_of_train' in locals():
            # clear local variable
            del station_highest_need_of_train

        backup_station: Station
        if 'backup_station' in locals():
            # clear local variable
            del backup_station

        for station in stations:
            if station.get_remaining_capacity() > 0:
                # Skip stations without enough train capacities

                # Calculate sum of passenger capacities of all Trains at current Station
                sum_capacities = 0
                for t_id in station.get_trains_in_station():
                    sum_capacities = sum_capacities + next(
                        (t.get_capacity() for t in trains if t.get_id() == t_id)
                        , 0)
                # Calculate sum of passenger sizes at current Station
                sum_passengers = 0
                for p_id in station.get_waiting_passengers_in_station():
                    sum_passengers = sum_passengers + next(
                        (p.get_group_size() for p in passengers if p.get_id() == p_id)
                        , 0)
                # Calculate difference between passenger capacities of all Trains and Passengers at current Station
                # => The higher the difference, the greater the need for an additional Train to handle the Passengers
                diff_passengers_capacities = sum_passengers - sum_capacities
                # Determine whether current Station has higher need for Train than other Stations
                # 1. No Train needed in a Station without passengers
                # 2. Train should start in Station where difference of Passengers and train capacities is highest
                if diff_passengers_capacities >= biggest_diff_passengers_capacities :
                    if station.get_waiting_passengers_in_station() != 0:
                        # Station has capacity and need of another Train
                        station_highest_need_of_train = station
                        biggest_diff_passengers_capacities = diff_passengers_capacities
                    else:
                        # Station has capacity but no need for a Train because it has no Passengers.
                        # Saving Station as backup
                        backup_station = station

        try:
            # Check if a starting Station was found.
            # If no Station was found, accessing the variable will throw a NameError exception
            station_highest_need_of_train

        except NameError:
            # No Station in need of another train was found
            try:
                # Check if any Backup-Station was found, that simply has enough capacities to take another train.
                # If no Backup-Station was found, accessing the variable will throw a NameError exception
                backup_station

            except NameError:
                # There is no Station that has enough capacities for an additional Train
                # This can only happen if the input data is
                file_log.write('\n\tERROR: There is no Station that has enough capacities for an additional Train.')
                print('\tERROR: There is no Station that has enough capacities for an additional Train.')
                break

            else:
                # The Backup-Station is selected as starting Station for the Train
                # even though it has no need for another Train to handle its Passengers.
                print('\tAusnahmesituation')
                station_highest_need_of_train = backup_station

        else:
            # Current Train without starting Station now starts at previously determined Station
            # with the highest difference between Passengers and passenger capacities of Trains at the Station
            train.set_starting_station(station_highest_need_of_train.get_id())
            station_highest_need_of_train.train_arrives(train.get_id())
            file_log.write('\nTrain ' + str(train.get_id()) + ': Starting Station = ' + str(train.get_current_location_id()))
            #print('Train ', train.get_id(), ': Starting Station = ', train.get_current_location_id())

        if train.get_start_station_id() == '*':
             file_log.write('\nERROR: Can not find starting station for train ' + str(train.get_id()))
             print('ERROR: Can not find starting station for train ', train.get_id())

    # Determine for each Station all the Lines that start/end on iter
    slowest_train: Train = trains[-1]
    longest_route = 0
    for station in stations:
        for line in lines:
            if line.get_first_station() == station.get_id() \
                    or line.get_second_station() == station.get_id():
                station.add_line_to_lines_from_station(line.get_id())
        # Determine longest route for slowest train
        prev_nodes, path_costs = dijkstra_algorithm(graph = slowest_train.get_graph(), start_node = station.get_id())
        longest_route_current_station = max(path_costs.values())
        if longest_route_current_station > longest_route:
            longest_route = longest_route_current_station

    maximal_rounds = longest_route

    # Custom rule for input where deadlocks can not be determined by looking at the latest expected arrival
    if os.path.basename(file_input.name).__contains__('custom_min_') and maximal_rounds < 1000:
        maximal_rounds = 1000

    file_log.write('\nMaximal rounds before emergency break of endless loop because of deadlock: '
            + str(maximal_rounds)
            + '\n(2 x latest expected arrival of a passenger)')
    print('Maximum rounds before emergency break of endless loop because of deadlock:', maximal_rounds
          , '\n(2 x latest expected arrival of a passenger)')

    maximal_rounds = 1000

    return maximal_rounds

def set_route_for_train(train: Train):
    """
    Determine and set route for passed Train

    :param train:
    :return:
    """
    priority_station_id: str = train.get_current_location_id()
    shortest_expected_arrival: int = sys.maxsize
    passenger_ids_on_train: List[str] = train.get_passengers_on_train()

    try:
        passenger_highest_need: Passenger = next(p for p in passengers
                                             if p.get_end_station_id() != p.get_current_location_id() \
                                                 and p.get_current_location_id().startswith('S')
                                                 and p.get_group_size() <= train.get_remaining_capacity())

    except StopIteration:
        # No Passenger needs a pick up
        if len(passenger_ids_on_train) == 0:
            # No Passengers on Train
            # and no Passengers need a pick up
            train.set_route_manually([])
            train.set_priority_station_id('')
            train.set_next_station_id('')
            file_log.write('\nTrain ' + train.get_id() + ': No Passengers on Train and no Passengers to be picked up')
            return
        file_log.write('\nTrain ' + train.get_id() + ': No Passenger to be picked up')

    for p in [ps for ps
              in passengers
              if ps.get_end_station_id() != ps.get_current_location_id()
              ]:
        # Iterate through passengers that are not already at their destination
        if p.get_expected_arrival() < shortest_expected_arrival \
                and p.get_id() in passenger_ids_on_train:
            # Found Passenger on Train that has higher need
            shortest_expected_arrival = p.get_expected_arrival()
            priority_station_id = p.get_end_station_id()

            # Für Performance Boost
            # break
        elif len(train.get_passengers_on_train()) == 0 and p.get_current_location_id().startswith('S'):
            # Train has no Passengers
            # Passenger is at Station but not on the current train
            if p.get_expected_arrival() < passenger_highest_need.get_expected_arrival() \
                    and p.get_group_size() <= train.get_remaining_capacity():
                # Passenger on another Station has higher need than the ones on Train
                # and Train has enough capacity to get him
                passenger_highest_need = p

                # Für Performance Boost
                # break

    if len(passenger_ids_on_train) == 0:
        # No passengers on train
        priority_station_id = passenger_highest_need.get_current_location_id()
        file_log.write('\nTrain ' + train.get_id() + ': No Passengers, drive to Passenger with highest need: '
                       + passenger_highest_need.get_id() + ' (' + priority_station_id + ')')
    else:
        file_log.write('\nTrain ' + train.get_id() + ': Passengers on board ' + str(train.get_passengers_on_train()) + ', drive to Passenger with highest need on Train: '
                       + priority_station_id)
    train.set_priority_station_id(priority_station_id)
    train.set_route()

def print_current_situation():
    """
    Prints all Stations, Lines, Trains and Passengers with additional information like current location or remaining capacity.

    For Passengers it also displays if they already arrived at their desired destination.

    :return:
    """
    for station in stations:
        file_log.write('\nStation ' + str(station.get_id()) + ': Trains = ' + str(station.get_trains_in_station()) + ', Remaining Capacity = ' + str(station.get_remaining_capacity()) + ', Passengers = ' + str(station.get_waiting_passengers_in_station()) + ', Lines = ' + str(station.get_lines_from_station()))
        #print('Station ', station.get_id(), ': Trains =', station.get_trains_in_station(), ', Remaining Capacity =', station.get_remaining_capacity(), ', Passengers =', station.get_waiting_passengers_in_station(), ', Lines =', station.get_lines_from_station())

    for line in lines:
        file_log.write('\nLine ' + str(line.get_id()) + ': Trains = ' + str(line.get_trains_on_line()) + ', Remaining Capacity = ' + str(line.get_remaining_capacity()) + ', Connecting Stations = ' + str([line.get_first_station(), line.get_second_station()]) + ', Length = ' + str(line.get_length()))
    #    print('Line ', line.get_id(), ': Trains =', line.get_trains_on_line(), ', Remaining Capacity =', line.get_remaining_capacity(), ', Connecting Stations =', [line.get_first_station(), line.get_second_station()], ', Length =', line.get_length())

    for train in trains:
        file_log.write('\nTrain ' + str(train.get_id()) + ': Location =' + str(train.get_current_location_id()) + ' (' + str(train.get_time_on_current_location()) + ' Round(s))' + ' Passengers =' + str(train.get_passengers_on_train()) + ', Destination =' + str(train.get_priority_station_id()) + ', Remaining Capacity =' + str(train.get_remaining_capacity()))

    for passenger in passengers:
        if passenger.get_current_location_id() == passenger.get_end_station_id():
            file_log.write('\n[X] Passenger ' + str(passenger.get_id()) + ': Delay = ' + str(passenger.get_delay()) + ', Location = ' + str(passenger.get_current_location_id()) + ', Destination = ' + str(passenger.get_end_station_id()) + ', Size = ' + str(passenger.get_group_size()) + ', Time = ' + str(passenger.get_expected_arrival()))#, ' (since ', passenger.get_time_on_current_location(), ' Rounds)')
    #        print('[X] Passenger ', passenger.get_id(), ': Delay =', passenger.get_delay(), ', Current location =', passenger.get_current_location_id(), ', Goal =', passenger.get_end_station_id(), ', Size =', passenger.get_group_size(), ', Time =', passenger.get_expected_arrival())#, ' (since ', passenger.get_time_on_current_location(), ' Rounds)')
        else:
            file_log.write('\n[ ] Passenger ' + str(passenger.get_id()) + ', Current Location = ' + str(passenger.get_current_location_id()) + ', Destination = ' + str(passenger.get_end_station_id()) + ', Size = ' + str(passenger.get_group_size()) + ', Expected Arrival = ' + str(passenger.get_expected_arrival()))#, ' (since ', passenger.get_time_on_current_location(), ' Rounds)')
    #        print('[ ] Passenger ', passenger.get_id(), ': Location =', passenger.get_current_location_id(), ', Destination =', passenger.get_end_station_id(), ', Size =', passenger.get_group_size(), ', Expected Arrival =', passenger.get_expected_arrival())#, ' (since ', passenger.get_time_on_current_location(), ' Rounds)')

def check_capacities():
    """
    Number of Trains on each Line must not exceed its capacities.

    Number of Passengers on each Train must not exceed its capacities.

    Number of Trains at each Station must not exceed its capacities

    :return: True if capacities are met. False if capacities are exceeded.
    """
    f = False

    # Number of Trains on each Line must not exceed its capacities
    if any(len(line.get_trains_on_line()) > line.get_capacity() for line in lines):
        f = True
        print('\nERROR: Too many Trains on the same Line')

    # Number of Passengers on each Train must not exceed its capacities
    for train in trains:
        passenger_ids = train.get_passengers_on_train()
        temp_passengers = filter(lambda p: p.get_id() in passenger_ids, passengers)
        p_group_sizes = map(lambda p: p.get_group_size(), temp_passengers)
        sum_passengers = sum(p_group_sizes)
        if sum_passengers > train.get_capacity():
            print('\nERROR: The number of passengers on train ', train.get_id(), ' exceed its capacities!')
            f = True
            break

    # Number of Trains at each Station must not exceed its capacities
    if any(len(station.get_trains_in_station()) > station.get_capacity() for station in stations):
        f = True
        print('\nERROR: Too many trains in the same station')

    return f

def check_ending_conditions(passenger_list):
    """
    Checks if all Passengers arrived at their desired destination.

    :return: True if all Passengers arrived at their final destination. Otherwise returns False.
    """
    result = True

    # All Passengers have to have arrived at their destination
    if any(pa.get_current_location_id() != pa.get_end_station_id() for pa in passenger_list):
        result = False

    return result

def prepare_output(train_list, passenger_list):
    """
    Initializes dictionaries for Passengers and Trains, where their actions are logged to.
    Later used to create the output file from.

    :param train_list: List of all Train-objects
    :param passenger_list: List of all Passenger-objects
    :return: Dictionary of Trains and dictionary of Passengers
    """
    t_dict = {}
    p_dict = {}

    for t in train_list:
        # Train logs are initialized with their starting station
        t_dict[t.get_id()] = '0 Start ' + t.get_current_location_id() + '\n'
    for pa in passenger_list:
        # Passengers logs are initialized with an empty string
        p_dict[pa.get_id()] = ''

    return t_dict, p_dict

# Execution

In [6]:
#---------------------------------------------------------------------
#----------------------------- Execution -----------------------------
#---------------------------------------------------------------------

# Uncomment the variant to use
input_files = [x for x in os.listdir('./Input')]
#input_files = [x for x in os.listdir('./Input') if x.startswith('custom_min')]
#input_files = [x for x in os.listdir('./Input') if not x.startswith('custom_min')]
#input_files = ['custom_min_57_10-27_12_20_30_14-934_5_10_4-143_20_input.txt']
for file_path_input in input_files:
    try:
        file_path_input = 'Input/' + file_path_input
        file_path_output = file_path_input.replace('Input', 'Output/' + notebook_name).replace('input', 'output')
        folder_path_log = "Log/" + notebook_name
        file_path_log = file_path_input.replace('Input', folder_path_log).replace('input', 'log')
        Path('Log/' + notebook_name).mkdir(parents=True, exist_ok=True)

        print('Start')
        print('\n--------------------- Preparation ---------------------')

        # Delete log file if it already exists:
        #print('Opening log file')
        if file_path_log.replace(folder_path_log + '/', '') in os.listdir(folder_path_log):
            print('File', file_path_log, 'already exists. Old version will be deleted.\n')
            os.remove(file_path_log)
        file_log = open(file_path_log, "a")
        #print('Opened log file')

        # Read input from file
        #print('Reading input file')
        file_input = open(file_path_input, "r")
        list_of_lines_input = file_input.readlines()   # reading text from file line-by-line into a list of strings
        # 2) Read input from filepath passed through command line:
        #list_of_lines_input = sys.stdin.readlines()

        unprocessed_data = list_of_lines_input
        #print('Finished reading input file')

        # Clean up, convert and split data for further processing
        #print('Data preprocessing')
        stations_df, lines_df, trains_df, passengers_df = data_preprocessing(unprocessed_data)
        #print('Finished data preprocessing')

        # Create train graphs
        #print('Creating train graphs')
        train_graphs = create_train_graphs()
        #print('Finished creating train graphs')

        # Map DataFrames from input to lists of objects
        #print('Mapping input to objects')
        stations, lines, trains, passengers = map_df_to_objects(stations_df, lines_df, trains_df, passengers_df)
        #print('Finished mapping input to objects')

        # ------------------- Algorithm -------------------

        # Preparation
        print('Preparation')
        max_rounds = preparation()
        print('Finished preparation')

        #Prepare standard output
        trains_dict, passengers_dict = prepare_output(trains, passengers)

        print('\n--------------------- Starting Algorithm ---------------------')
        print('Input:')
        print(len(stations_df), '\tStations')
        print(len(lines_df), '\tLines')
        print(len(trains_df), '\tTrains')
        print(len(passengers_df), '\tPassengers')
        print()
        print('Earliest expected arrival:', min(p.get_expected_arrival() for p in passengers))
        print('Latest expected arrival:', max(p.get_expected_arrival() for p in passengers))

        # Trains are blocked for the rest of the round, if they
        # arrived, started or have not finished on their current Line yet
        blocked_train_ids: [str] = []
        waiting_train_ids: [str] = []

        # Initialize round counter
        current_round = 0
        inactive_counter = 1

        # Endless loop, realizing round based approach.
        # Fulfilling the end or abort conditions cancels the loop.
        while True:
            # Increment round counter
            current_round = current_round + 1

            file_log.write('\n--------------------- Round ' + str(current_round) + ' ---------------------')
            #print('\n--------------------- Round ', current_round, ' ---------------------')

            # Clear lists of blocked and waiting trains each round
            blocked_train_ids = []
            waiting_train_ids = []

            # ------------------- Current Situation -------------------
            # This increases runtime significantly, it should only be used to debug small inputs
            #print_current_situation()

            # ------------------- Actions -------------------
            #print('\n-- Actions --\n')
            file_log.write('\n-- Actions --\n')

            # Passenger activities
            # First let Passengers detrain if on final destination
            for passenger in [p for p in passengers if p.get_current_location_id() != p.get_end_station_id()]:
                passenger_current_location = next((station for station in stations if station.get_id() == passenger.get_current_location_id()), '')
                if type(passenger_current_location) is not Station:
                    passenger_current_location = next((train for train in trains if train.get_id() == passenger.get_current_location_id()), '')

                if type(passenger_current_location) is Train:
                    # Passengers is on Train
                    # possible actions: leaving the Train

                    # passenger_current_location = Train Object
                    # train_current_location_id = Line or Station ID
                    # train_arrival_station = Station Object
                    train_current_location_id = passenger_current_location.get_current_location_id()

                    train_current_location = next((station for station in stations if station.get_id() == train_current_location_id), '')

                    if train_current_location_id in [station.get_id() for station in stations] \
                            and type(train_current_location) == Station \
                            and passenger.get_end_station_id() == train_current_location_id:

                        # Train is in Station which is the Passengers final destination
                        # Passenger detrains / leaves Train
                        #print(passenger.get_end_station_id(), passenger.get_current_location_id())
                        passenger.detrain(train_current_location, current_round)
                        #print(passenger.get_end_station_id(), passenger.get_current_location_id())
                        #file_log.write('\nPassenger ' + str(passenger.get_id()) + ' leaves Train ' + str(train.get_id()) + ' on Station ' + )
                        passengers_dict[passenger.get_id()] += str(current_round) + ' Detrain\n'
                        if passenger_current_location.get_id() not in blocked_train_ids:
                            blocked_train_ids.append(passenger_current_location.get_id())
                        #print('Passenger', passenger.get_id(), 'leaves Train', passenger_current_location.get_id(), 'on Station', train_arrival_station.get_id())
                    # else: Train just departed from a Station or it is not the Passengers final destination
                    continue

            # Second let Passengers board
            # Passengers are always sorted by their expected arrival from early to late
            for passenger in [p for p in passengers if p.get_current_location_id() != p.get_end_station_id()]:
                # determine if the Passenger is in Station or on Train
                passenger_current_location = next((station for station in stations if station.get_id() == passenger.get_current_location_id()), '')
                if type(passenger_current_location) is not Station:
                    passenger_current_location = next((train for train in trains if train.get_id() == passenger.get_current_location_id()), '')

                if type(passenger_current_location) is Station:
                    # Passenger is in Station
                    # possible actions: boarding a Train

                    # Passenger is not at its destination and wants to board a Train
                    # Get all Trains in current Station
                    # The list should still be sorted by their speed from fast to slow
                    trains_in_current_station = [t for t in trains if t.get_id() in passenger_current_location.get_trains_in_station()]

                    train_to_board: Train = None
                    shortest_duration: int = sys.maxsize - 1
                    total_delay_train_passengers: int = sys.maxsize
                    total_duration_diff_train_passengers: int = sys.maxsize
                    delay_comparison: int = sys.maxsize
                    for train in trains_in_current_station:
                        # Check which Train gets the Passenger to its destination the fastest
                        total_duration_diff_train_passengers: int = sys.maxsize
                        total_delay_train_passengers: int = sys.maxsize
                        temp_duration: int = sys.maxsize
                        if (passenger.get_group_size() + train.get_passenger_utilization()) > train.get_capacity():
                            # Train has no capacity for the Passenger group
                            continue
                        if len(train.get_passengers_on_train()) == 0:
                            # Train has no Passengers so current Passenger will be the VIP
                            temp_duration = train.calculate_duration_fastest_route(end_station_id=passenger.get_end_station_id())
                            total_duration_diff_train_passengers = 0
                            total_delay_train_passengers = 0
                        else:
                            # Train already has Passengers
                            if len(train.get_route()) > 0:
                                # Train has an active route
                                if passenger.get_end_station_id() in train.get_route():
                                    # Current Passengers' destination is part of the route
                                    temp_duration = train.calculate_duration_current_route(end_station_id=passenger.get_end_station_id())
                                else:
                                    # Current Passengers' destination is NOT part of the route
                                    # But maybe after unloading the current VIP, the new route goes to his destination
                                    temp_duration, total_duration_diff_train_passengers, total_delay_train_passengers = train.calculate_duration_multiple_routes(
                                        end_station_id=passenger.get_end_station_id()
                                        , expected_arrival=passenger.get_expected_arrival()
                                        , current_passenger_id=passenger.get_id()
                                    )
                            else:
                                # Train has Passengers but no active route
                                # That means the other Passengers also have just boarded
                                temp_duration, total_duration_diff_train_passengers, total_delay_train_passengers = train.calculate_duration_multiple_routes(
                                    end_station_id=passenger.get_end_station_id()
                                    , expected_arrival=passenger.get_expected_arrival()
                                    , current_passenger_id=passenger.get_id()
                                )
                        if total_delay_train_passengers < delay_comparison:
                                delay_comparison = total_delay_train_passengers
                                shortest_duration = temp_duration
                                train_to_board = train
                        elif total_delay_train_passengers == delay_comparison:
                            if temp_duration <= shortest_duration:
                                delay_comparison = total_delay_train_passengers
                                train_to_board = train
                                shortest_duration = temp_duration

                    if shortest_duration != sys.maxsize - 1:
                        passenger.board(train_to_board)
                        passengers_dict[passenger.get_id()] += str(current_round) + ' Board ' + train_to_board.get_id() + '\n'
                        file_log.write('\nPassenger ' + str(passenger.get_id()) + ' boards on Train ' + str(train_to_board.get_id()))
                        #print('Passenger', passenger.get_id() , 'boards on Train', train.get_id())
                        if train_to_board.get_id() not in blocked_train_ids:
                            blocked_train_ids.append(train_to_board.get_id())

                    if passenger.get_current_location_id() == passenger_current_location.get_id() \
                            and passenger.get_current_location_id() != passenger.get_end_station_id():
                        # Passenger stays in Station
                        file_log.write('\nPassenger ' + str(passenger.get_id())
                              + ' stays in Station ' + str(passenger_current_location.get_id())
                              + ' because there is no Train that has enough capacity and drives to his desired Station.')
                        #print('Passenger', passenger.get_id()
                        #      , 'stays in Station', passenger_current_location.get_id()
                        #      , 'because there is no Train that has enough capacity and drives to his desired Station.')
                        continue

                    continue
                elif not type(passenger_current_location) is Train:
                    # Error case
                    file_log.write('\nERROR: Passenger ' + str(passenger.get_id()) + ' is neither on a Train nor in a Station! Where is he???\n')
                    print('\nERROR: Passenger', passenger.get_id(), 'is neither on a Train nor in a Station! Where is he???\n')
                    continue

            # print()
            # for train in [t for t in sorted(sorted(trains, key=lambda y: len(y.get_passengers_on_train()), reverse=True), key=lambda x: min([pas.get_expected_arrival() for pas in passengers if pas.get_id() in x.get_passengers_on_train()], default=0), reverse=False)]:
            #     print(train.get_id(), len(train.get_passengers_on_train()), min([pas.get_expected_arrival() for pas in passengers if pas.get_id() in train.get_passengers_on_train()], default=0))
            # print()

            # Train activities for Trains with Passengers
            # Trains are sorted by their Passengers' expected arrival from early to late
            for train in [t for t in sorted(sorted(trains, key=lambda y: len(y.get_passengers_on_train()), reverse=True), key=lambda x: min([pas.get_expected_arrival() for pas in passengers if pas.get_id() in x.get_passengers_on_train()], default=0), reverse=False)]:
                if train.get_id() in blocked_train_ids or len(train.get_passengers_on_train()) == 0:
                    #print('Skipped Train', train.get_id(), 'because it already had its action (switched location with an other Train or Passengers boarded).')
                    continue

                # determine if Train is in Station or on Line
                train_current_location = next((station for station in stations if station.get_id() == train.get_current_location_id()), '')
                if type(train_current_location) is not Station:
                    train_current_location = next((line for line in lines if line.get_id() == train.get_current_location_id()), '')

                if type(train_current_location) is Station:
                    # Train is in Station
                    # possible actions: starting on Line, waiting in Station

                    lines_in_current_station = [line for line in lines if (line.get_first_station() == train_current_location.get_id() or line.get_second_station() == train_current_location.get_id())]

                    #if len(train.get_route()) == 0:
                        # determine the route for the Train according to the Passengers priorities
                    set_route_for_train(train)

                    # Train tries to depart on a Line
                    for line in lines_in_current_station:
                        # Train looks for a line that brings it to the next Station on its route

                        if line.get_first_station() == train_current_location.get_id():
                            line_next_station_id = line.get_second_station()
                        else:
                            line_next_station_id = line.get_first_station()

                        if len(train.get_route()) > 0 \
                                and line_next_station_id == train.get_route()[0] :
                            # Train has Passengers and wants to depart on this Line

                            if len(line.get_trains_on_line()) < line.get_capacity() :
                                # Train departs on Line (Line has enough capacities)
                                train.depart_on_line(line)
                                trains_dict[train.get_id()] += str(current_round) + ' Depart ' + line.get_id() +'\n'
                                if train.get_id() not in blocked_train_ids:
                                    blocked_train_ids.append(train.get_id())
                                file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' starts on Line ' + str(line.get_id()))

                                break # next Train

                            else:
                                # The Line has not enough capacity for the Train to start on it without further action
                                # if there is a Train on the Line, that wants to arrive in this Station too,
                                # capacities can be exceeded temporarily to exchange both Trains

                                file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' wants to start on Line ' + str(line.get_id()) + ' but has to wait, until another Train is leaving the Line.')
                                file_log.write('\n\tBlocked Trains: ' + str(blocked_train_ids))
                                file_log.write('\n\tTrains on Line ' + str(line.get_id()) + ': ' + str(line.get_trains_on_line()))

                                exchange_train: Train
                                if 'exchange_train' in locals():
                                    # clear local variable
                                    del exchange_train

                                # find Train (not blocked) that wants to arrive in this Station
                                for tr in trains:
                                    if tr.get_current_location_id() == line.get_id() \
                                            and tr.get_id() not in blocked_train_ids \
                                            and tr.get_next_station_id() == train_current_location.get_id() \
                                            and (tr.get_time_on_current_location() + 1) * tr.get_speed() >= line.get_length():
                                        exchange_train = tr
                                        break

                                try:
                                    # Check if a Train for the exchange was found.
                                    # If no Train was found, accessing the variable will throw a NameError exception
                                    exchange_train

                                except NameError:
                                    # There is no train that wants to arrive at this station

                                    file_log.write('\n\tThere is no Train that wants to arrive at this Station in this round')

                                else:
                                    # Exchange Trains between Station and Line

                                    #print('\tFound a Train to exchange locations:', exchange_train.get_id(), '(', exchange_train.get_current_location_id(), ')')

                                    # 1) Train on Line leaves Line and arrives in Station
                                    exchange_train.arrive_in_station(train_current_location)
                                    if exchange_train.get_id() not in blocked_train_ids:
                                        blocked_train_ids.append(exchange_train.get_id())

                                    file_log.write('\nTrain ' + str(exchange_train.get_id()) + ': ' + str(train.get_route()) + ' leaves Line ' + str(line.get_id()) + ' arriving in Station ' + str(train_current_location.get_id()))

                                    # 2) Train in Station leaves Station and starts on Line.
                                    # Train is not blocked, because it can arrive at the next station if it is fast enough.
                                    train.depart_on_line(line)
                                    trains_dict[train.get_id()] += str(current_round) + ' Depart ' + line.get_id() +'\n'

                                    file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' starts on Line ' + str(line.get_id()))

                                break # next Train

                        else:
                            # Line has not enough capacities
                            continue # next Line

                    if train.get_current_location_id() == train_current_location.get_id():
                        # Train waits in Station
                        # the time on the current location is not incremented here (which would result in blocking)
                        # because the Train can still have another action like exchanging with an incoming Train
                        file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' waits in Station ' + str(train_current_location.get_id()) + ', route ' + str(train.get_route()) + ', passengers ' + str(train.get_passengers_on_train()))

                        if train.get_id() not in waiting_train_ids:
                            waiting_train_ids.append(train.get_id())

                # again determine if Train is on a Line now, because a Train can start on and leave a Line
                # in the same round if he is fast enough
                train_current_location = next((line for line in lines if line.get_id() == train.get_current_location_id()), '')
                if type(train_current_location) is not Line:
                    train_current_location = next((station for station in stations if station.get_id() == train.get_current_location_id()), '')

                if type(train_current_location) is Line:
                    # Train is on Line
                    # possible actions: leaving the Line, continuing on the Line

                    train_arrival_station = next(station for station in stations if station.get_id() == train.get_next_station_id())

                    if (train.get_time_on_current_location() + 1) * train.get_speed() >= train_current_location.get_length():
                        # Train has reached the end of the Line and wants to arrive in Station
                        if train_arrival_station.get_remaining_capacity() > 0:
                            # Train leaves Line (Station has enough capacity)

                            file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' leaves Line ' + str(train_current_location.get_id()) + ' arriving in Station ' + str(train_arrival_station.get_id()))

                            train.arrive_in_station(train_arrival_station)
                            if train.get_id() not in blocked_train_ids:
                                blocked_train_ids.append(train.get_id())
                            continue # next Train

                        else:
                            # the Station has not enough capacity for the Train to arrive in it without further action
                            # if there is a Train in the Station, that wants to depart on this Line too,
                            # capacities can be exceeded temporarily to exchange both Trains

                            file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' wants to arrive in Station ' + str(train_arrival_station.get_id()) + ' but has to wait, until another Train is leaving the Station.')
                            file_log.write('\n\tBlocked Trains: ' + str(blocked_train_ids))
                            file_log.write('\n\tTrains in Station ' + str(train_arrival_station.get_id()) + ': ' + str(train_arrival_station.get_trains_in_station()))

                            exchange_train: Train
                            if 'exchange_train' in locals():
                                # clear local variable
                                del exchange_train

                            # determine the Station that is connected by by the current Line
                            if train_current_location.get_first_station() == train.get_next_station_id():
                                line_other_station_id = train_current_location.get_second_station()
                            else:
                                line_other_station_id = train_current_location.get_first_station()

                            # find Train (not blocked) that has no passengers and could depart from this Station
                            for tr in trains:
                                if tr.get_current_location_id() == train_arrival_station.get_id() \
                                        and tr.get_id() not in blocked_train_ids \
                                        and len(tr.get_passengers_on_train()) == 0:
                                    exchange_train = tr
                                    break

                            # find Train (not blocked) that wants to depart from this Station
                            for tr in trains:
                                if tr.get_current_location_id() == train_arrival_station.get_id() \
                                        and tr.get_id() not in blocked_train_ids \
                                        and tr.get_next_station_id() == line_other_station_id:
                                    exchange_train = tr
                                    break

                            try:
                                # check if a Train for the exchange was found.
                                # If no train was found, accessing the variable will throw a NameError exception
                                exchange_train

                            except NameError:
                                # There is no train that wants to arrive at this station

                                file_log.write('\n\tThere is no Train that wants to depart from this Station in this round')
                                file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' waits on Line ' + str(train_current_location.get_id()) + ' until another Train leaves Station ' + str(train_arrival_station.get_id()))
                                if train.get_id() not in waiting_train_ids:
                                    waiting_train_ids.append(train.get_id())
                                continue # next Train

                            else:
                                # Exchange Trains between Station and Line

                                file_log.write('\n\tFound a Train to exchange locations: ' + str(exchange_train.get_id()) + ' (' + str(exchange_train.get_current_location_id()) + ')')

                                if len(exchange_train.get_passengers_on_train()) == 0 \
                                       and len(exchange_train.get_route()) > 0 \
                                       and line_other_station_id != exchange_train.get_next_station_id():
                                   # If exchange Train has no Passengers but already has a route and the exchange line is not part of it
                                   # add the exchange line forth an back to the route of the empty train
                                   exchange_train.set_route_manually([line_other_station_id, exchange_train.get_current_location_id()] + exchange_train.get_route())

                                # 1) Train in Station leaves Station and starts on Line
                                exchange_train.depart_on_line(train_current_location)
                                trains_dict[exchange_train.get_id()] += str(current_round) + ' Depart ' + train_current_location.get_id() +'\n'
                                if exchange_train.get_id() not in blocked_train_ids:
                                    blocked_train_ids.append(exchange_train.get_id())
                                #print('Train', exchange_train.get_id() , 'starts on Line', train_current_location.get_id())

                                # 2) Train on Line leaves Line and arrives in Station
                                train.arrive_in_station(train_arrival_station)
                                if train.get_id() not in blocked_train_ids:
                                    blocked_train_ids.append(train.get_id())
                                #print('Train', train.get_id() , 'leaves Line', train_current_location.get_id(), 'arriving in Station', train_arrival_station.get_id())

                        # else:
                        #     # Train waits on Line, because there is no capacity in the Station yet
                        #     train.increment_time_on_current_location()
                        #     print('Train', train.get_id() , 'waits on Line', train_current_location.get_id(), 'because there is no capacity in Station', train_arrival_station.get_id())

                    else:
                        # Train continues on Line (blocked), because the has not reached the end yet
                        train.increment_time_on_current_location()
                        if train.get_id() not in blocked_train_ids:
                            blocked_train_ids.append(train.get_id())

                        file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' drives on Line ' + str(train_current_location.get_id()))
                        continue # next Train

                if type(train_current_location) is not Line and type(train_current_location) is not Station:
                    # Error case
                    file_log.write('\nERROR: Train ' + str(train.get_id()) + ' is neither on a Line nor in a Station (' + str(type(train.get_current_location_id())) + ')! Where is it???\n')
                    print('\nERROR: Train', train.get_id(), 'is neither on a Line nor in a Station (', type(train.get_current_location_id()) , ')! Where is it???\n')
                    continue # next Train
            # Train activities for Trains without Passengers
            # Trains are sorted by their speed from fast to slow
            for train in trains:#[t for t in sorted(trains, key=lambda x: min([pas.get_expected_arrival() for pas in passengers if (pas.get_id() in x.get_passengers_on_train())], default=0), reverse=False)]:
                if train.get_id() in blocked_train_ids or len(train.get_passengers_on_train()) > 0:
                    #print('Skipped Train', train.get_id(), 'because it already had its action (switched location with an other Train or Passengers boarded).')
                    continue

                # determine if Train is in Station or on Line
                train_current_location = next((station for station in stations if station.get_id() == train.get_current_location_id()), '')
                if type(train_current_location) is not Station:
                    train_current_location = next((line for line in lines if line.get_id() == train.get_current_location_id()), '')

                if type(train_current_location) is Station:
                    # Train is in Station
                    # possible actions: starting on Line, waiting in Station

                    lines_in_current_station = [line for line in lines if (line.get_first_station() == train_current_location.get_id() or line.get_second_station() == train_current_location.get_id())]

                    #if len(train.get_route()) == 0:
                        # determine the route for the Train according to the Passengers priorities
                    set_route_for_train(train)

                    # Train tries to depart on a Line
                    for line in lines_in_current_station:
                        # Train looks for a line that brings it to the next Station on its route

                        if line.get_first_station() == train_current_location.get_id():
                            line_next_station_id = line.get_second_station()
                        else:
                            line_next_station_id = line.get_first_station()

                        if len(train.get_route()) > 0 \
                                and line_next_station_id == train.get_route()[0] :
                            # Train has Passengers and wants to depart on this Line

                            if len(line.get_trains_on_line()) < line.get_capacity() :
                                # Train departs on Line (Line has enough capacities)
                                train.depart_on_line(line)
                                trains_dict[train.get_id()] += str(current_round) + ' Depart ' + line.get_id() +'\n'
                                if train.get_id() not in blocked_train_ids:
                                    blocked_train_ids.append(train.get_id())
                                file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' starts on Line ' + str(line.get_id()))
                                break # next Train

                            else:
                                # the Line has not enough capacity for the Train to start on it without further action
                                # if there is a Train on the Line, that wants to arrive in this Station too,
                                # capacities can be exceeded temporarily to exchange both Trains

                                file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' wants to start on Line ' + str(line.get_id()) + ' but has to wait, until another Train is leaving the Line.')
                                file_log.write('\n\tBlocked Trains: ' + str(blocked_train_ids))
                                file_log.write('\n\tTrains on Line ' + str(line.get_id()) + ': ' + str(line.get_trains_on_line()))

                                exchange_train: Train
                                if 'exchange_train' in locals():
                                    # clear local variable
                                    del exchange_train

                                # find Train (not blocked) that wants to arrive in this Station
                                for tr in trains:
                                    if tr.get_current_location_id() == line.get_id() \
                                            and tr.get_id() not in blocked_train_ids \
                                            and tr.get_next_station_id() == train_current_location.get_id() \
                                            and (tr.get_time_on_current_location() + 1) * tr.get_speed() >= line.get_length():
                                        exchange_train = tr
                                        break

                                try:
                                    # Check if a Train for the exchange was found.
                                    # If no Train was found, accessing the variable will throw a NameError exception
                                    exchange_train

                                except NameError:
                                    # There is no train that wants to arrive at this station

                                    file_log.write('\n\tThere is no Train that wants to arrive at this Station in this round')

                                else:
                                    # Exchange Trains between Station and Line

                                    #print('\tFound a Train to exchange locations:', exchange_train.get_id(), '(', exchange_train.get_current_location_id(), ')')

                                    # 1) Train on Line leaves Line and arrives in Station
                                    exchange_train.arrive_in_station(train_current_location)
                                    if exchange_train.get_id() not in blocked_train_ids:
                                        blocked_train_ids.append(exchange_train.get_id())

                                    file_log.write('\nTrain ' + str(exchange_train.get_id()) + ': ' + str(train.get_route()) + ' leaves Line ' + str(line.get_id()) + ' arriving in Station ' + str(train_current_location.get_id()))

                                    # 2) Train in Station leaves Station and starts on Line.
                                    # Train is not blocked, because it can arrive at the next station if it is fast enough.
                                    train.depart_on_line(line)
                                    trains_dict[train.get_id()] += str(current_round) + ' Depart ' + line.get_id() +'\n'

                                    file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' starts on Line ' + str(line.get_id()))

                                break # next Train

                        else:
                            # Line has not enough capacities
                            continue # next Line

                    if train.get_current_location_id() == train_current_location.get_id():
                        # Train waits in Station
                        # the time on the current location is not incremented here (which would result in blocking)
                        # because the Train can still have another action like exchanging with an incoming Train
                        file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' waits in Station ' + str(train_current_location.get_id()) + ', route ' + str(train.get_route()) + ', passengers ' + str(train.get_passengers_on_train()))

                        if train.get_id() not in waiting_train_ids:
                            waiting_train_ids.append(train.get_id())

                # again determine if Train is on a Line now, because a Train can start on and leave a Line
                # in the same round if he is fast enough
                train_current_location = next((line for line in lines if line.get_id() == train.get_current_location_id()), '')
                if type(train_current_location) is not Line:
                    train_current_location = next((station for station in stations if station.get_id() == train.get_current_location_id()), '')

                if type(train_current_location) is Line:
                    # Train is on Line
                    # possible actions: leaving the Line, continuing on the Line

                    train_arrival_station = next(station for station in stations if station.get_id() == train.get_next_station_id())

                    if (train.get_time_on_current_location() + 1) * train.get_speed() >= train_current_location.get_length():
                        # Train has reached the end of the Line and wants to arrive in Station
                        if train_arrival_station.get_remaining_capacity() > 0:
                            # Train leaves Line (Station has enough capacity)

                            file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' leaves Line ' + str(train_current_location.get_id()) + ' arriving in Station ' + str(train_arrival_station.get_id()))

                            train.arrive_in_station(train_arrival_station)
                            if train.get_id() not in blocked_train_ids:
                                blocked_train_ids.append(train.get_id())
                            continue # next Train

                        else:
                            # the Station has not enough capacity for the Train to arrive in it without further action
                            # if there is a Train in the Station, that wants to depart on this Line too,
                            # capacities can be exceeded temporarily to exchange both Trains

                            file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' wants to arrive in Station ' + str(train_arrival_station.get_id()) + ' but has to wait, until another Train is leaving the Station.')
                            file_log.write('\n\tBlocked Trains: ' + str(blocked_train_ids))
                            file_log.write('\n\tTrains in Station ' + str(train_arrival_station.get_id()) + ': ' + str(train_arrival_station.get_trains_in_station()))

                            exchange_train: Train
                            if 'exchange_train' in locals():
                                # clear local variable
                                del exchange_train

                            # determine the Station that is connected by by the current Line
                            if train_current_location.get_first_station() == train.get_next_station_id():
                                line_other_station_id = train_current_location.get_second_station()
                            else:
                                line_other_station_id = train_current_location.get_first_station()

                            # find Train (not blocked) that has no passengers and could depart from this Station
                            for tr in trains:
                                if tr.get_current_location_id() == train_arrival_station.get_id() \
                                        and tr.get_id() not in blocked_train_ids \
                                        and len(tr.get_passengers_on_train()) == 0:
                                    exchange_train = tr
                                    break

                            # find Train (not blocked) that wants to depart from this Station
                            for tr in trains:
                                if tr.get_current_location_id() == train_arrival_station.get_id() \
                                        and tr.get_id() not in blocked_train_ids \
                                        and tr.get_next_station_id() == line_other_station_id:
                                    exchange_train = tr
                                    break

                            try:
                                # check if a Train for the exchange was found.
                                # If no train was found, accessing the variable will throw a NameError exception
                                exchange_train

                            except NameError:
                                # There is no train that wants to arrive at this station

                                file_log.write('\n\tThere is no Train that wants to depart from this Station in this round')
                                file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' waits on Line ' + str(train_current_location.get_id()) + ' until another Train leaves Station ' + str(train_arrival_station.get_id()))
                                if train.get_id() not in waiting_train_ids:
                                    waiting_train_ids.append(train.get_id())
                                continue # next Train

                            else:
                                # Exchange Trains between Station and Line

                                file_log.write('\n\tFound a Train to exchange locations: ' + str(exchange_train.get_id()) + ' (' + str(exchange_train.get_current_location_id()) + ')')

                                if len(exchange_train.get_passengers_on_train()) == 0 \
                                       and len(exchange_train.get_route()) > 0 \
                                       and line_other_station_id != exchange_train.get_next_station_id():
                                   # If exchange Train has no Passengers but already has a route and the exchange line is not part of it
                                   # add the exchange line forth an back to the route of the empty train
                                   exchange_train.set_route_manually([line_other_station_id, exchange_train.get_current_location_id()] + exchange_train.get_route())

                                # 1) Train in Station leaves Station and starts on Line
                                exchange_train.depart_on_line(train_current_location)
                                trains_dict[exchange_train.get_id()] += str(current_round) + ' Depart ' + train_current_location.get_id() +'\n'
                                if exchange_train.get_id() not in blocked_train_ids:
                                    blocked_train_ids.append(exchange_train.get_id())
                                #print('Train', exchange_train.get_id() , 'starts on Line', train_current_location.get_id())

                                # 2) Train on Line leaves Line and arrives in Station
                                train.arrive_in_station(train_arrival_station)
                                if train.get_id() not in blocked_train_ids:
                                    blocked_train_ids.append(train.get_id())
                                #print('Train', train.get_id() , 'leaves Line', train_current_location.get_id(), 'arriving in Station', train_arrival_station.get_id())

                        # else:
                        #     # Train waits on Line, because there is no capacity in the Station yet

                    else:
                        # Train continues on Line (blocked), because the has not reached the end yet
                        train.increment_time_on_current_location()
                        if train.get_id() not in blocked_train_ids:
                            blocked_train_ids.append(train.get_id())

                        file_log.write('\nTrain ' + str(train.get_id()) + ': ' + str(train.get_route()) + ' drives on Line ' + str(train_current_location.get_id()))

                        continue # next Train

                if type(train_current_location) is not Line and type(train_current_location) is not Station:
                    # Error case
                    file_log.write('\nERROR: Train ' + str(train.get_id()) + ' is neither on a Line nor in a Station (' + str(type(train.get_current_location_id())) + ')! Where is it???\n')
                    print('\nERROR: Train', train.get_id(), 'is neither on a Line nor in a Station (', type(train.get_current_location_id()) , ')! Where is it???\n')
                    continue # next Train

            # For all Trains that had no actions the time on the current location has to be incremented
            for train_id in waiting_train_ids:
                # Train stays in station
                current_train = next(t for t in trains if t.get_id() == train_id)

                file_log.write('\nTrain ' + str(train_id) + ': ' + str(current_train.get_route()) + ' stays on Location ' + str(current_train.get_current_location_id()))
                current_train.increment_time_on_current_location()
                # no need to add or remove train from lists because they get reset each round

            file_log.write('\n')
            #print()

            # Check abort conditions (capacities exceeded at the end of a round)
            failed = check_capacities()

            # Check ending conditions ()
            finished = check_ending_conditions(passengers)

            # Cancel endless loop if conditions are met and show results
            if finished or failed:
                file_log.write('\n-------------------------------------------------------------')
                file_log.write('\n--------------------- End on Round ' + str(current_round) + ' ---------------------')
                file_log.write('\n-------------------------------------------------------------')
                print() # Print empty line
                print('\n-------------------------------------------------------------')
                print('--------------------- End on Round ', current_round, ' ---------------------')
                print('-------------------------------------------------------------')
                if failed:
                    # Capacities have been exceeded at the end of the round
                    file_log.write('\nEnded successfully: No')
                    print('Ended successfully: No')
                else:
                    # Successfully finished execution
                    # Calculate total delay and cancel loop
                    file_log.write('\nEnded successfully: Yes')
                    print('Ended successfully: Yes')
                    total_delay = 0
                    for p in passengers:
                        total_delay = total_delay + p.get_delay()
                    file_log.write('\nTotal delay:' + str(total_delay) + '\n')
                    print('Total delay:', total_delay, '\n')
                    print_current_situation()
                break
            else:
                #print() # Print empty line
                if current_round > max_rounds:
                #if current_round > 30:
                    # Maximum number of rounds have been reached, meaning the algorithm has run into a deadlock.
                    # Cancel execution
                    file_log.write('\n--------------------- End on Round ' + str(current_round) + ' ---------------------')
                    file_log.write('\nEnd condition met: No (possible deadlock)')
                    print('\n--------------------- End on Round ', current_round, ' ---------------------')
                    print('End condition met: No (possible deadlock)')
                    break

        file_log.close()
        file_input.close()

        # Output

        folder_path_output = "Output/" + notebook_name
        Path(folder_path_output).mkdir(parents=True, exist_ok=True)

        # delete file if it already exists:
        if file_path_output.replace(folder_path_output + '/', '') in os.listdir(folder_path_output):
            print('File', file_path_output, 'already exists. Old version will be deleted.\n')
            os.remove(file_path_output)

        file_output = open(file_path_output, "a")

        # Write output to file:
        for key, value in trains_dict.items():
            file_output.write('[Train:' + key + ']\n')
            #print('[Train: ' + key + ']')
            file_output.write(value)
            #print(value)
            file_output.write('\n')

        for key, value in passengers_dict.items():
            file_output.write('[Passenger:' + key + ']\n')
            #print('[Passenger:' + key + ']')
            file_output.write(value)
            #print(value)
            file_output.write('\n')

        print('Finished creating output-file', file_path_output)
        file_output.close()

    except Exception as ex:
        # Output in case of an error
        traceback.print_exc()
        folder_path_output = "Output/" + notebook_name
        Path(folder_path_output).mkdir(parents=True, exist_ok=True)

        # delete file if it already exists:
        if file_path_output.replace(folder_path_output + '/', '') in os.listdir(folder_path_output):
            print('File', file_path_output, 'already exists. Old version will be deleted.\n')
            os.remove(file_path_output)

        file_output = open(file_path_output, "a")

        # write output to file:
        file_output.write('ERROR')

        print('Finished creating output-file for error case', file_path_output, '\n', ex)
        file_output.close()
        continue

print('\n\n------------------- Done with all -------------------')

Start

--------------------- Preparation ---------------------
File Log/main/capacity_log.txt already exists. Old version will be deleted.

Preparation
Maximum rounds before emergency break of endless loop because of deadlock: 2 
(2 x latest expected arrival of a passenger)
Finished preparation

--------------------- Starting Algorithm ---------------------
Input:
2 	Stations
1 	Lines
5 	Trains
3 	Passengers

Earliest expected arrival: 4
Latest expected arrival: 4


-------------------------------------------------------------
--------------------- End on Round  3  ---------------------
-------------------------------------------------------------
Ended successfully: Yes
Total delay: 0 

File Output/main/capacity_output.txt already exists. Old version will be deleted.

Finished creating output-file Output/main/capacity_output.txt
Start

--------------------- Preparation ---------------------
File Log/main/custom_min_11_10-8_12_20_30_4-31_5_10_4-15_20_log.txt already exists. Old version

In [7]:
file_log.close()